# Simple Graph

In [19]:
import org.apache.spark.graphx.{Edge, EdgeRDD, Graph, VertexRDD}
import org.apache.spark.sql.{functions => F}

import org.apache.spark.sql.{functions=>F}


## Create Vertexs

In [1]:
val vertexIDs = sc.parallelize(Array(1,2,3,4,5,6,7))

vertexIDs = ParallelCollectionRDD[0] at parallelize at <console>:27


ParallelCollectionRDD[0] at parallelize at <console>:27

In [8]:
val vertexRDD = vertexIDs.map(id => (id.toLong, "Person_" + id))

vertexRDD = MapPartitionsRDD[16] at map at <console>:30


MapPartitionsRDD[16] at map at <console>:30

## Create Edges

In [12]:
val edgeRDD = sc.parallelize(Array(Edge(1L,2L,4), Edge(1L,3L,2),
                                   Edge(1L,4L,5), Edge(3L,4L,1),
                                   Edge(5L,6L,3), Edge(5L,7L,1), Edge(6L,7L,3)))

edgeRDD = ParallelCollectionRDD[33] at parallelize at <console>:28


ParallelCollectionRDD[33] at parallelize at <console>:28

## Create Graph

In [13]:
val graph = Graph(vertexRDD, edgeRDD)

graph = org.apache.spark.graphx.impl.GraphImpl@4d8f59f3


org.apache.spark.graphx.impl.GraphImpl@4d8f59f3

## Check degrees

In [14]:
graph.degrees.collect()

[(1,3), (2,1), (3,2), (4,2), (5,2), (6,2), (7,2)]

In [15]:
graph.degrees.map(_._2).stats()

(count: 7, mean: 2.000000, stdev: 0.534522, max: 3.000000, min: 1.000000)

In [20]:
graph.degrees.innerJoin(graph.vertices){
    case(id, degree, name) => (id, name, degree)
}.values.toDF("id", "name", "degree").sort(F.desc("degree")).show()

+---+--------+------+
| id|    name|degree|
+---+--------+------+
|  1|Person_1|     3|
|  7|Person_7|     2|
|  6|Person_6|     2|
|  3|Person_3|     2|
|  5|Person_5|     2|
|  4|Person_4|     2|
|  2|Person_2|     1|
+---+--------+------+



## Connected Componets

In [21]:
val connectedComponentsGraph = graph.connectedComponents()

connectedComponentsGraph = org.apache.spark.graphx.impl.GraphImpl@593f1737


org.apache.spark.graphx.impl.GraphImpl@593f1737

In [24]:
val componentDF = connectedComponentsGraph.vertices.toDF("vid", "cid")

componentDF = [vid: bigint, cid: bigint]


[vid: bigint, cid: bigint]

In [25]:
componentDF.show()

+---+---+
|vid|cid|
+---+---+
|  1|  1|
|  2|  1|
|  3|  1|
|  4|  1|
|  5|  5|
|  6|  5|
|  7|  5|
+---+---+



In [26]:
componentDF.groupBy("cid").count().show()

+---+-----+
|cid|count|
+---+-----+
|  5|    3|
|  1|    4|
+---+-----+

